In [1]:
# Import Statements

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
import pickle
from xgboost import XGBClassifier

In [2]:
# Step 1 : Load data set
data = pd.read_csv('credit_card_data.csv')
data = data.drop(['ID'], axis=1)

null_counts = data.isnull().sum()
print("Missing Values Count : " ,null_counts)

print("\nColumn Names:")
print(data.columns.tolist())

print("Number of Data Points")
print(data.shape)

Missing Values Count :  LIMIT_BAL    0
SEX          0
EDUCATION    0
MARRIAGE     0
AGE          0
PAY_0        0
PAY_2        0
PAY_3        0
PAY_4        0
PAY_5        0
PAY_6        0
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
BILL_AMT4    0
BILL_AMT5    0
BILL_AMT6    0
PAY_AMT1     0
PAY_AMT2     0
PAY_AMT3     0
PAY_AMT4     0
PAY_AMT5     0
PAY_AMT6     0
DEFAULT      0
dtype: int64

Column Names:
['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'DEFAULT']
Number of Data Points
(30000, 24)


In [3]:
# Step 2 : Scaling and One Hot Encoding
num_c = ['LIMIT_BAL', 'AGE', 
                'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3','BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 
                'PAY_AMT1','PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6' ]
cat_c = ['SEX', 'MARRIAGE']

data[num_c] = StandardScaler().fit_transform(data[num_c])
data = pd.get_dummies(data, columns=cat_c, drop_first=True)

bool_c = data.select_dtypes(include=['bool']).columns

# Convert them to int (True → 1, False → 0)
data[bool_c] = data[bool_c].astype(int)
print(data.iloc[0])


LIMIT_BAL    -1.136720
EDUCATION     2.000000
AGE          -1.246020
PAY_0         2.000000
PAY_2         2.000000
PAY_3        -1.000000
PAY_4        -1.000000
PAY_5        -2.000000
PAY_6        -2.000000
BILL_AMT1    -0.642501
BILL_AMT2    -0.647399
BILL_AMT3    -0.667993
BILL_AMT4    -0.672497
BILL_AMT5    -0.663059
BILL_AMT6    -0.652724
PAY_AMT1     -0.341942
PAY_AMT2     -0.227086
PAY_AMT3     -0.296801
PAY_AMT4     -0.308063
PAY_AMT5     -0.314136
PAY_AMT6     -0.293382
DEFAULT       1.000000
SEX_2         1.000000
MARRIAGE_1    1.000000
MARRIAGE_2    0.000000
MARRIAGE_3    0.000000
Name: 0, dtype: float64


In [4]:
# Step 3 : Train and Split
y = data['DEFAULT']
X = data.drop('DEFAULT', axis=1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, stratify=y, random_state = 17)

In [5]:
# Create model
xgb_model = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    eval_metric='logloss'
)

In [6]:
# Train
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [7]:
with open("xg_boost_model.pkl", "wb") as file:
    pickle.dump(xgb_model, file)

In [8]:
# Predict
y_pred = xgb_model.predict(X_test)

In [9]:
# Accuracy of the classifier

acc = accuracy_score(y_test, y_pred)
print(acc * 100 , "%")

81.66666666666667 %


In [10]:
# Step 6 : Build the confusion matrix

conf_matrix = confusion_matrix(y_test, y_pred) 
print('Confusion Matrix :')
print(conf_matrix)

Confusion Matrix :
[[4421  252]
 [ 848  479]]


In [11]:
# Step 7 : Classification Report
print('Classification Report: ')
print(classification_report(y_test, y_pred))

Classification Report: 
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      4673
           1       0.66      0.36      0.47      1327

    accuracy                           0.82      6000
   macro avg       0.75      0.65      0.68      6000
weighted avg       0.80      0.82      0.80      6000



In [12]:
# Step 8 : AUC Score
y_prob = xgb_model.predict_proba(X_test)[:,1]
auc_score = roc_auc_score(y_test, y_prob)

print("AUC:", auc_score)

AUC: 0.7814842790866288
